<h1>Section 1: This section obtains data from the wikipedia page and stores it appropriately in a dataframe</h1>

In [7]:
import pandas as pd

In [8]:
aslist = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [3]:
type(aslist)
len(aslist)

3

In [9]:
df = aslist[0]
df = df[df['Borough'] != 'Not assigned']
df['Neighborhood'] = df['Neighborhood'].str.replace(' / ', ',')
df.reset_index()
df = df[['Postal code','Borough', 'Neighborhood']]
df.reset_index(drop=True,inplace=True)
df

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park,Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern,Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill,Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
df.shape

(103, 3)

<h1>Section 2: In this section, we read latitude/longitude data and add it to our dataframe from Section 1</h1>

In [10]:
df3 = pd.read_csv('http://cocl.us/Geospatial_data')
df3.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df.rename(columns={"Postal code":"PostalCode"}, inplace=True)
df3.rename(columns={"Postal Code":"PostalCode"}, inplace=True)
df3.columns

Index(['PostalCode', 'Latitude', 'Longitude'], dtype='object')

In [12]:
out = pd.merge(df.reset_index(), df3.reset_index())

In [13]:
out.head()

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,28,M3H,North York,"Bathurst Manor,Wilson Heights,Downsview North",43.754328,-79.442259
1,41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,88,M8V,Etobicoke,"New Toronto,Mimico South,Humber Bay Shores",43.605647,-79.501321


<h1>Section 3: In this section, we will analyse neighborhoods in Toronto and cluster them</h1>

In [2]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: \ ^C
failed

CondaError: KeyboardInterrupt



In [1]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [ ]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [ ]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [15]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[43.651070, -79.347015], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(out['Latitude'], out['Longitude'], out['Borough'], out['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [16]:
CLIENT_ID = 'DXOLZNK5E3HB4GQLOFBTMK1KZKZVADQFLFXVXZHRQOHBXKTK' # your Foursquare ID
CLIENT_SECRET = 'UGSJTKYNZOXNZCYPIK0XP2UTADUJZQVFU43VEJDWBALFRSF4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DXOLZNK5E3HB4GQLOFBTMK1KZKZVADQFLFXVXZHRQOHBXKTK
CLIENT_SECRET:UGSJTKYNZOXNZCYPIK0XP2UTADUJZQVFU43VEJDWBALFRSF4


In [17]:
neighborhood_latitude = out.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = out.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = out.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bathurst Manor,Wilson Heights,Downsview North are 43.7543283, -79.4422593.


In [18]:
# type your answer here
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e85b04895feaf001b5ca38e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Clanton Park',
  'headerFullLocation': 'Clanton Park, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 20,
  'suggestedBounds': {'ne': {'lat': 43.7588283045, 'lng': -79.43604092869812},
   'sw': {'lat': 43.749828295499995, 'lng': -79.44847767130189}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4adf35f1f964a520577821e3',
       'name': "Wolfie's Deli",
       'location': {'address': '670 Sheppard Ave. W',
        'crossStreet': 'at Bryant St.',
        'lat': 43.754875481443555,
        'lng': -79.44243785404784,
        'labeledLatLngs': [{

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wolfie's Deli,Deli / Bodega,43.754875,-79.442438
1,Bagel Plus,Restaurant,43.755395,-79.440686
2,Orly Restaurant & Grill,Middle Eastern Restaurant,43.754493,-79.443507
3,Best for Bride in Toronto,Bridal Shop,43.755789,-79.437834
4,Starbucks,Coffee Shop,43.755703,-79.440483


In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

20 venues were returned by Foursquare.


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
# type your answer here

toronto_venues = getNearbyVenues(names=out['Neighborhood'],
                                   latitudes=out['Latitude'],
                                   longitudes=out['Longitude']
                                  )



Bathurst Manor,Wilson Heights,Downsview North
The Danforth West,Riverdale
New Toronto,Mimico South,Humber Bay Shores


In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(74, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Bathurst Manor,Wilson Heights,Downsview North",43.754328,-79.442259,Wolfie's Deli,43.754875,-79.442438,Deli / Bodega
1,"Bathurst Manor,Wilson Heights,Downsview North",43.754328,-79.442259,Bagel Plus,43.755395,-79.440686,Restaurant
2,"Bathurst Manor,Wilson Heights,Downsview North",43.754328,-79.442259,Orly Restaurant & Grill,43.754493,-79.443507,Middle Eastern Restaurant
3,"Bathurst Manor,Wilson Heights,Downsview North",43.754328,-79.442259,Best for Bride in Toronto,43.755789,-79.437834,Bridal Shop
4,"Bathurst Manor,Wilson Heights,Downsview North",43.754328,-79.442259,Starbucks,43.755703,-79.440483,Coffee Shop


In [26]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor,Wilson Heights,Downsview North",20,20,20,20,20,20
"New Toronto,Mimico South,Humber Bay Shores",12,12,12,12,12,12
"The Danforth West,Riverdale",42,42,42,42,42,42


In [27]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 43 uniques categories.


In [28]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,Bakery,Bank,Bookstore,Brewery,Bridal Shop,Bubble Tea Shop,Café,Caribbean Restaurant,...,Pizza Place,Pub,Restaurant,Sandwich Place,Shopping Mall,Spa,Supermarket,Sushi Restaurant,Trail,Yoga Studio
0,"Bathurst Manor,Wilson Heights,Downsview North",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Bathurst Manor,Wilson Heights,Downsview North",0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,"Bathurst Manor,Wilson Heights,Downsview North",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Bathurst Manor,Wilson Heights,Downsview North",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Bathurst Manor,Wilson Heights,Downsview North",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
toronto_onehot.shape

(74, 44)

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,Bakery,Bank,Bookstore,Brewery,Bridal Shop,Bubble Tea Shop,Café,Caribbean Restaurant,...,Pizza Place,Pub,Restaurant,Sandwich Place,Shopping Mall,Spa,Supermarket,Sushi Restaurant,Trail,Yoga Studio
0,"Bathurst Manor,Wilson Heights,Downsview North",0.000000,0.000000,0.1,0.000000,0.00000,0.05,0.00000,0.000000,0.00000,...,0.050000,0.00000,0.050000,0.05,0.05,0.00000,0.05,0.05,0.00000,0.00000
1,"New Toronto,Mimico South,Humber Bay Shores",0.083333,0.083333,0.0,0.000000,0.00000,0.00,0.00000,0.083333,0.00000,...,0.083333,0.00000,0.083333,0.00,0.00,0.00000,0.00,0.00,0.00000,0.00000
2,"The Danforth West,Riverdale",0.023810,0.023810,0.0,0.047619,0.02381,0.00,0.02381,0.023810,0.02381,...,0.023810,0.02381,0.023810,0.00,0.00,0.02381,0.00,0.00,0.02381,0.02381


In [32]:
toronto_grouped.shape

(3, 44)

In [33]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor,Wilson Heights,Downsview North----
            venue  freq
0            Bank  0.10
1     Coffee Shop  0.10
2      Restaurant  0.05
3  Sandwich Place  0.05
4     Pizza Place  0.05


----New Toronto,Mimico South,Humber Bay Shores----
                 venue  freq
0  American Restaurant  0.08
1           Restaurant  0.08
2               Bakery  0.08
3         Liquor Store  0.08
4   Mexican Restaurant  0.08


----The Danforth West,Riverdale----
                    venue  freq
0        Greek Restaurant  0.21
1             Coffee Shop  0.10
2      Italian Restaurant  0.07
3  Furniture / Home Store  0.05
4               Bookstore  0.05




In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor,Wilson Heights,Downsview North",Coffee Shop,Bank,Convenience Store,Ice Cream Shop,Gas Station,Deli / Bodega,Bridal Shop,Diner,Middle Eastern Restaurant,Fried Chicken Joint
1,"New Toronto,Mimico South,Humber Bay Shores",American Restaurant,Pizza Place,Fast Food Restaurant,Gym,Liquor Store,Mexican Restaurant,Pet Store,Pharmacy,Café,Restaurant
2,"The Danforth West,Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Ice Cream Shop,Furniture / Home Store,Brewery,Bubble Tea Shop,Café,Caribbean Restaurant


In [38]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 1], dtype=int32)

In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = out

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,28,M3H,North York,"Bathurst Manor,Wilson Heights,Downsview North",43.754328,-79.442259,0,Coffee Shop,Bank,Convenience Store,Ice Cream Shop,Gas Station,Deli / Bodega,Bridal Shop,Diner,Middle Eastern Restaurant,Fried Chicken Joint
1,41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Ice Cream Shop,Furniture / Home Store,Brewery,Bubble Tea Shop,Café,Caribbean Restaurant
2,88,M8V,Etobicoke,"New Toronto,Mimico South,Humber Bay Shores",43.605647,-79.501321,2,American Restaurant,Pizza Place,Fast Food Restaurant,Gym,Liquor Store,Mexican Restaurant,Pet Store,Pharmacy,Café,Restaurant


In [44]:
# create map
map_clusters = folium.Map(location=[43.651070, -79.347015], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3H,-79.442259,0,Coffee Shop,Bank,Convenience Store,Ice Cream Shop,Gas Station,Deli / Bodega,Bridal Shop,Diner,Middle Eastern Restaurant,Fried Chicken Joint


In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4K,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Ice Cream Shop,Furniture / Home Store,Brewery,Bubble Tea Shop,Café,Caribbean Restaurant


In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M8V,-79.501321,2,American Restaurant,Pizza Place,Fast Food Restaurant,Gym,Liquor Store,Mexican Restaurant,Pet Store,Pharmacy,Café,Restaurant
